In [1]:
# check installation of python 
! which python

# check installation of pip
! which pip

# upgrade pip if needed 
! pip install --upgrade pip

# install biopython
! pip install biopython

# upgrade biopython if needed 

! pip install biopython --upgrade

# import biopython and check import
import Bio
print(Bio.__version__)

/Users/giana.cirolia/anaconda3/bin/python
/Users/giana.cirolia/anaconda3/bin/pip
     |████████████████████████████████| 1.5 MB 632 kB/s eta 0:00:0104
  Attempting uninstall: pip
    Found existing installation: pip 20.3.3
    Uninstalling pip-20.3.3:
      Successfully uninstalled pip-20.3.3
  Using cached biopython-1.78-cp36-cp36m-macosx_10_9_x86_64.whl (2.2 MB)
     |████████████████████████████████| 2.3 MB 1.9 MB/s eta 0:00:01     |█████████████▉                  | 972 kB 269 kB/s eta 0:00:05
1.78


In [2]:
## import command line interface for biopython blasts 
## more documentation here https://biopython.org/docs/1.75/api/Bio.Blast.Applications.html
from Bio.Blast.Applications import NcbiblastnCommandline

In [63]:
! mkdir -p database/output
cline = NcbiblastnCommandline(query="sequences/queries/D.mel_query.fasta", db="database/genomes_db", strand="both",
                              evalue=0.001, out="database/output/D.mel_blast.xml", outfmt=5)
cline
print(cline)
cline()

blastn -out database/output/D.mel_blast.xml -outfmt 5 -query sequences/queries/D.mel_query.fasta -db database/genomes_db -evalue 0.001 -strand both


('', '')

In [67]:
## parsingg XML blast output for one test record 
## https://biopython.org/docs/1.75/api/Bio.Blast.Record.html

from Bio.Blast import NCBIXML

record_handle = open("database/output/test_blast.xml", 'r')

blast_records = NCBIXML.parse(record_handle)
blast_record = next(blast_records)

E_VALUE_THRESH = 1e-20 ## research this number for threshold we want

for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect< E_VALUE_THRESH:
            print("******Alignment******")
            #print("query: {}".format)
            print("sequence:", alignment.title) ## name of contig in file 
            print("length of contig: {}".format(alignment.length)) ## length of whole contig
            print("query_match_start_residue: {}, query_match_end_residue: {}".format(hsp.query_start, hsp.query_end))
            print("sequence match start: {}, sequence match end: {}".format(hsp.sbjct_start ,hsp.sbjct_end))
            print("e value:", hsp.expect)
            print(hsp.query[0:100]) ## search seq
            print(hsp.match[0:100]) ## lines connecting them
            print(hsp.sbjct[0:100]) ## contig matched to 
            




******Alignment******
sequence: contig_1--organism-Drosophila-mauritiana No definition line
length of contig: 4863
query_match_start_residue: 1, query_match_end_residue: 320
sequence match start: 1, sequence match end: 320
e value: 1.52349e-167
GATTCCGAGACAGAAAAGTCAACATGAGCCATACGCTCACCTGGTCTATACTGAATATCAAATTCGAAAGATTGCATGTACGACCACCACTCTATGAACT
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
GATTCCGAGACAGAAAAGTCAACATGAGCCATACGCTCACCTGGTCTATACTGAATATCAAATTCGAAAGATTGCATGTACGACCACCACTCTATGAACT
******Alignment******
sequence: contig_420--organism-Drosophila-mauritiana No definition line
length of contig: 309857
query_match_start_residue: 1, query_match_end_residue: 312
sequence match start: 239625, sequence match end: 239320
e value: 9.4992e-140
GATTCCGAGACAGAAAAGTCAACATGAGCCATACGCTCACCTGGTCTATACTGAATATCAAATTCGAAAGATTGCATGTACGACCACCACTCTATGAACT
|||||||||||||||| ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| ||||

In [66]:
E_VALUE_THRESH = 1e-20 
for record in NCBIXML.parse(open("database/output/D.mel_blast.xml", 'r')): 
     if record.alignments: 
        print("\n") 
        print("query: %s" % record.query[:100]) 
        for align in record.alignments: 
           for hsp in align.hsps: 
              if hsp.expect < E_VALUE_THRESH: 
                 print("match: %s " % align.title[:100])



query: dmel_stripe2_minimal_bronsky
match: contig_1--organism-Drosophila-melanogaster No definition line 
match: contig_529--organism-Drosophila-sechellia No definition line 
match: contig_203--organism-Drosophila-simulans No definition line 
match: contig_22--organism-Drosophila-mauritiana No definition line 
